In [2]:
import random
from data import ImageDetectionsField, TextField, RawField
from data import COCO, DataLoader
from data.dataset import AP_Dataset, APeval_Dataset, SA_Dataset, SAeval_Dataset
import evaluation
from evaluation import PTBTokenizer, Cider
# from models.transformer import Transformer, MemoryAugmentedEncoder, MeshedDecoder, ScaledDotProductAttentionMemory
from models.grid_m2_rst import  Transformer, TransformerEncoder, MeshedDecoder, ScaledDotProductAttention
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.nn import NLLLoss
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader as TorchDataLoader
from torch.utils.data import Dataset as TorchDataset
import argparse, os, pickle
from tqdm import tqdm
import numpy as np
import itertools
import multiprocessing
from shutil import copyfile
import h5py
from utils import text_progress2, text_progress
import pandas as pd
import json

random.seed(1234)
torch.manual_seed(1234)
np.random.seed(1234)

In [3]:
def evaluate_metrics(model, dataloader, text_field, mode="multiple", is_sample=False, beam_size=5, top_k=5, top_p=0.8):
    import itertools
    print(dataloader)
    print(mode)
    model.eval()
    gen = {}
    gts = {}
    with tqdm(desc='evalulateion metrics', unit='it', total=len(dataloader)) as pbar:
        for it, batch in enumerate(iter(dataloader)):
            images = batch['roi_feat']
            caps_gt = batch['cap']
            images = images.to(device)
            with torch.no_grad():
#                 beam_size = 5
                out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<eos>'], beam_size, out_size=1, is_sample=is_sample, top_k=5, top_p=0.8)
#                 if decode == "beam_search":
#                     out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<eos>'], 5, out_size=1, is_sample=False)
#                 elif decode == "top-k_sampling":
#                     out, _ = model.beam_search(images, 20, text_field.vocab.stoi['<eos>'], 1, out_size=1, is_sample=True)
            caps_gen = text_field.decode(out, join_words=False)
            for i, (gts_i, gen_i) in enumerate(zip(caps_gt, caps_gen)):
                gen_i = ' '.join([k for k, g in itertools.groupby(gen_i)])
                gen['%d_%d' % (it, i)] = [gen_i, ]
                if mode == "multiple":
                    gts['%d_%d' % (it, i)] = gts_i
                elif mode == "single":
                    gts['%d_%d' % (it, i)] = [gts_i[0]]
            pbar.update()

    gts = evaluation.PTBTokenizer.tokenize(gts)
    gen = evaluation.PTBTokenizer.tokenize(gen)
    scores, _ = evaluation.compute_scores(gts, gen, spice=False)
    return scores

In [4]:
device = torch.device('cuda')

In [5]:
# Model
text_field = TextField(init_token='<bos>', eos_token='<eos>', lower=True, tokenize='spacy',
                       remove_punctuation=True, nopoints=False)
text_field.vocab = pickle.load(open('vocab.pkl', 'rb'))

encoder = TransformerEncoder(3, 0, attention_module=ScaledDotProductAttention, attention_module_kwargs={'m': 40})
decoder = MeshedDecoder(len(text_field.vocab), 54, 3, text_field.vocab.stoi['<pad>'])
model = Transformer(text_field.vocab.stoi['<bos>'], encoder, decoder).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 1. artpedia

In [6]:
train_myidx = np.load('../Dataset/artpedia/train_myidx.npy')
val_myidx = np.load('../Dataset/artpedia/val_myidx.npy')
test_myidx = np.load('../Dataset/artpedia/test_myidx.npy')

ap_train_dataset = h5py.File("../Dataset/artpedia/ap_train_grid.hdf5", "r")
ap_val_dataset = h5py.File("../Dataset/artpedia/ap_val_grid.hdf5", "r")
ap_test_dataset = h5py.File("../Dataset/artpedia/ap_test_grid.hdf5", "r")
print("loading data: done!!!")

loading data: done!!!


In [8]:
# artpedia dataset
dict_artpedia_test = APeval_Dataset(ap_test_dataset, test_myidx, text_field, max_detections=49, feature_type="grid", lower=True, remove_punctuation=True, tokenize='spacy')

# artpedia, dataloader
dict_artpedia_test_data_loader = TorchDataLoader(dict_artpedia_test, batch_size=50, collate_fn=lambda x: text_progress2(x))

#### 1.1 artpedia, one caption for evaluation, beam search

#### 1.2 artpedia, multiple captions for evaluation, beam search

In [9]:
# *** grid_m2rst
# without fine-tuning, off-the-shelf model
data = torch.load('saved_models_grid_m2rst/grid_m2_rst_best.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:09<00:00,  1.39s/it]


{'BLEU': [0.16722814874916128,
  0.0647967385843378,
  0.026314210811938347,
  0.013047885850321551],
 'METEOR': 0.04336997219011802,
 'ROUGE': 0.1567383673086055,
 'CIDEr': 0.020031510059636076}

In [10]:
# *** grid_m2rst
# without fine-tuning, off-the-shelf model
data = torch.load('saved_models_grid_m2rst/grid_m2_rst_last.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:09<00:00,  1.34s/it]


{'BLEU': [0.16766664243617874,
  0.06706854786854069,
  0.02652070869867405,
  0.013109732358008455],
 'METEOR': 0.04340685159778225,
 'ROUGE': 0.15892789987745576,
 'CIDEr': 0.02003948688173837}

In [11]:
# *** grid_m2rst
# without fine-tuning, off-the-shelf model
data = torch.load('saved_models_grid_m2rst/grid_m2_rst_epoch41.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:09<00:00,  1.33s/it]


{'BLEU': [0.16552490383500296,
  0.06339171836966011,
  0.024411996142851448,
  0.011465048096354524],
 'METEOR': 0.04304402204721149,
 'ROUGE': 0.15745674695633402,
 'CIDEr': 0.019602907170169808}

In [15]:
# fine-tune on artpedia, one image with one caption for training
data = torch.load('saved_models/artpedia_finetune_singlecap.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.94it/s]


{'BLEU': [0.2263770796898909,
  0.10189771635005827,
  0.04723688221258288,
  0.024825597404249923],
 'ROUGE': 0.2044362212159059,
 'CIDEr': 0.03947330064255181}

In [13]:
# *** grid_m2rst
# fine-tune on artpedia, one image with multiple captions for training
data = torch.load('saved_models/grid_m2_tr_last_18epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field)

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:09<00:00,  1.34s/it]


{'BLEU': [0.24697949865754792,
  0.12218483456845088,
  0.05717566420149227,
  0.030616814624825857],
 'METEOR': 0.06583208797468693,
 'ROUGE': 0.22393089972424116,
 'CIDEr': 0.039414558295047364}

In [16]:
# fine-tune on artpedia, one image with multiple captions for training
data = torch.load('saved_models/artpedia_finetune_mulcap.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field)

multiple


evalulateion metrics: 100%|██████████| 66/66 [00:13<00:00,  4.95it/s]


{'BLEU': [0.21496088729295892,
  0.10277365798269977,
  0.04822575058710956,
  0.024778542489085762],
 'ROUGE': 0.2091605244824866,
 'CIDEr': 0.03343695862148488}

In [22]:
# fine-tune on artpedia, one image with multiple captions for training      shuffle
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field)

multiple


evalulateion metrics: 100%|██████████| 329/329 [01:03<00:00,  5.18it/s]


{'BLEU': [0.23290164145038766,
  0.11628312648636245,
  0.052702077361140964,
  0.02689678159915748],
 'ROUGE': 0.21914811563939154,
 'CIDEr': 0.03148247854508848}

In [8]:
# beam search   remove unk
# fine-tune on artpedia, one image with multiple captions for training      shuffle
data = torch.load('saved_models/artpedia_finetune_mulcap_shuffle.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, is_sample=False)

multiple


evalulateion metrics: 100%|██████████| 329/329 [01:05<00:00,  5.00it/s]


{'BLEU': [0.33960716243253547,
  0.16180978856929548,
  0.0747107343165195,
  0.035472046837868663],
 'ROUGE': 0.22451221642237176,
 'CIDEr': 0.06145132500410184}

In [10]:
# beam search   remove unk
# fine-tune on artpedia, one image with multiple captions for training      shuffle
data = torch.load('saved_models_grid_m2rst_apft/grid_m2_tr_last_18epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_artpedia_test_data_loader, text_field, is_sample=False)

multiple


evalulateion metrics: 100%|██████████| 7/7 [00:12<00:00,  1.74s/it]


{'BLEU': [0.3609881069135,
  0.17902517899338843,
  0.09118977721377369,
  0.050181107544762536],
 'METEOR': 0.09133971685517936,
 'ROUGE': 0.23398104425547456,
 'CIDEr': 0.09330209254997349}

## 2. semart

In [6]:
sa_test_csv = pd.read_csv("../Dataset/SemArt/prediction_csvs/semart_test_prediction.csv")
sa_test_csv = sa_test_csv[sa_test_csv['predictioin']==0]
test_roi_feats = h5py.File("../Dataset/SemArt/sa_test_grid.hdf5", "r")
test_img_names = np.unique(sa_test_csv['img_name'].to_numpy())
test_img_caps_map = json.load(open('../Dataset/SemArt/test_img_caps_map.json'))

dict_semart_test = SAeval_Dataset(sa_test_csv, test_img_names, test_img_caps_map, test_roi_feats, text_field, max_detections=49, lower=True, remove_punctuation=True, tokenize='spacy')
dict_semart_test_data_loader = TorchDataLoader(dict_semart_test, batch_size=50,
                                  collate_fn=lambda x: text_progress2(x))

#### 2.1 semart, multiple captions for evaluation

In [6]:
# off-the-shelf model
data = torch.load('saved_models/saved_models_grid_m2rst/grid_m2_rst_last.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.73s/it]


{'BLEU': [0.1238723982265556,
  0.04377896610833391,
  0.017749577060783853,
  0.008296285553807443],
 'METEOR': 0.03853641775444528,
 'ROUGE': 0.1403943502673313,
 'CIDEr': 0.026411970140918662}

In [7]:
# off-the-shelf model
data = torch.load('saved_models/saved_models_grid_m2rst/grid_m2_rst_best.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.71s/it]


{'BLEU': [0.12809294283327557,
  0.045397722109473716,
  0.01791234797525151,
  0.007650384719347912],
 'METEOR': 0.039360563038510016,
 'ROUGE': 0.142495980976305,
 'CIDEr': 0.020916017474374143}

In [8]:
# off-the-shelf model
data = torch.load('saved_models/saved_models_grid_m2rst/grid_m2_rst_epoch41.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.70s/it]


{'BLEU': [0.13904005551409623,
  0.04929343466036093,
  0.019266916799502324,
  0.008159874882971943],
 'METEOR': 0.03898631586267387,
 'ROUGE': 0.13827194208741386,
 'CIDEr': 0.02001249028494681}

In [7]:
# fine-tune on SemArt    remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_last_17epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:25<00:00,  1.98s/it]


{'BLEU': [0.1793097313567625,
  0.08845628213335589,
  0.04550863303247926,
  0.027544256454569226],
 'METEOR': 0.059977900521655046,
 'ROUGE': 0.210684243568821,
 'CIDEr': 0.05792592281250193}

In [8]:
# fine-tune on SemArt          remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_best_13epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:17<00:00,  1.38s/it]


{'BLEU': [0.19366517108620626,
  0.09414067272111563,
  0.047131246404860994,
  0.02727318372337316],
 'METEOR': 0.06290828571071426,
 'ROUGE': 0.21849475404941598,
 'CIDEr': 0.0640887464734465}

In [9]:
# fine-tune on SemArt             remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_last_15epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:17<00:00,  1.38s/it]


{'BLEU': [0.20122972946628226,
  0.10130244427949057,
  0.05035051252524529,
  0.029524666351640177],
 'METEOR': 0.06385577952828095,
 'ROUGE': 0.21882635632016711,
 'CIDEr': 0.07495384156844488}

In [10]:
# fine-tune on SemArt            remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_last_16epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:17<00:00,  1.38s/it]


{'BLEU': [0.19248471632586958,
  0.09910268608348924,
  0.0526078570669242,
  0.03235305832917967],
 'METEOR': 0.06283993741113443,
 'ROUGE': 0.2192800786090442,
 'CIDEr': 0.0693193711305087}

In [9]:
# fine-tune on SemArt    remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_last_17epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.73s/it]


{'BLEU': [0.3072302366961828,
  0.15600789631516362,
  0.07752334709721519,
  0.04534257500040296],
 'METEOR': 0.08315250711965914,
 'ROUGE': 0.23498774340842055,
 'CIDEr': 0.09807077016422536}

In [10]:
# fine-tune on SemArt          remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_best_13epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.74s/it]


{'BLEU': [0.31320652324860604,
  0.15754597536164425,
  0.0777770804175481,
  0.04421370018307126],
 'METEOR': 0.08476561809644537,
 'ROUGE': 0.23779380640853018,
 'CIDEr': 0.09827848953660719}

In [11]:
# fine-tune on SemArt             remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_last_15epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.74s/it]


{'BLEU': [0.31100768669817086,
  0.1571050036064454,
  0.07634610949345182,
  0.04242987274654262],
 'METEOR': 0.08418395341362825,
 'ROUGE': 0.23695722537609906,
 'CIDEr': 0.09888653207449181}

In [12]:
# fine-tune on SemArt            remove <unk>
data = torch.load('saved_models/saved_models_saft_grid_m2rst/sa_gridm2rst_sa_last_16epoch.pth')
model.load_state_dict(data['state_dict'])
evaluate_metrics(model, dict_semart_test_data_loader, text_field, mode='multiple')

multiple


evalulateion metrics: 100%|██████████| 13/13 [00:22<00:00,  1.74s/it]


{'BLEU': [0.3125313141354439,
  0.15712767945367956,
  0.076700577593618,
  0.043780874821767166],
 'METEOR': 0.0846857648929584,
 'ROUGE': 0.23493355890015902,
 'CIDEr': 0.09656533432493654}